## Assymetric Cryptography

In [216]:
import random
import math
import codecs
import base64

In [217]:
class Utils:
    def __init__(self):
        '''if not used then delete'''
    
    def hex_to_dec(hex):
        deci = int(hex, 16)
        return deci

    def dec_to_hex(dec):
        hexa = hex(dec)
        return hexa

    def hex_to_b64(hex):
        hex = hex[2:] if(len(hex[2:]) % 2 == 0) else '0' + hex[2:]
        b64 = codecs.encode(codecs.decode(hex, 'hex'), 'base64').decode()
        return b64
       
    def b64_to_hex(b64):
        hexa = base64.b64decode(b64).hex()
        return hexa
    
    def dec_to_b64(dec):
        b64 = Utils.hex_to_b64(Utils.dec_to_hex(dec))
        return b64

    def b64_to_dec(b64):
        deci = Utils.hex_to_dec(Utils.b64_to_hex(b64))
        return deci

    def is_square(i):
        return i == math.isqrt(i) ** 2

    def mod_sqrt(n, p):

        points = []
        n = n % p
        for x in range (1, p):
            if (pow(x, 2, p) == n):
                points.append(x)
        return points

        

In [218]:

class ElGamal:
    def __init__(self):
        pass

    def create_key(self, p, g, x):
        y = pow(g, x, p)
        public_key = { 'y': y, 'g': g, 'p': p }
        private_key = { 'x': x, 'p': p }
        return public_key, private_key

    def encrypt(self, message, key, public_key):
        a = pow(public_key['g'], key, public_key['p'])
        b = (pow(public_key['y'], key, public_key['p']) * (message % public_key['p'])) % public_key['p']
        return { 'a': a, 'b': b }

    def decrypt(self, message, private_key):
        return ((message['b'] % private_key['p']) * pow(message['a'], (private_key['p'] - 1 - private_key['x']), private_key['p'])) % private_key['p']


In [219]:
elgamal = ElGamal()

public_key, private_key = elgamal.create_key(2357, 2, 1751)

print("Public Key\t: " + str(public_key))
print("Private Key\t: " + str(private_key))

encrypted = elgamal.encrypt(2035, 1520, public_key)
print("Ciphertext\t: " + str(encrypted))

decrypted = elgamal.decrypt(encrypted, private_key)
print("Plaintext\t: " + str(decrypted))

Public Key	: {'y': 1185, 'g': 2, 'p': 2357}
Private Key	: {'x': 1751, 'p': 2357}
Ciphertext	: {'a': 1430, 'b': 697}
Plaintext	: 2035


In [220]:
class ElGamalMachine:
    def __init__(self):
        self.elgamal = ElGamal()
    
    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_key(self, bit):
        p = self.create_random_prime(bit)
        g = random.randint(1, p - 1)
        x = random.randint(1, p -2)

        public_key, private_key = self.elgamal.create_key(p, g, x)

        public_key['y'] = Utils.dec_to_b64(public_key['y'])
        public_key['g'] = Utils.dec_to_b64(public_key['g'])
        public_key['p'] = Utils.dec_to_b64(public_key['p'])

        private_key['x'] = Utils.dec_to_b64(private_key['x'])
        private_key['p'] = Utils.dec_to_b64(private_key['p'])

        pub_key = public_key['y'] + public_key['g'] + public_key['p']
        pri_key = private_key['x'] + private_key['p']

        return pub_key, pri_key

    def create_key_file(self, bit):
        public_key, private_key = self.create_key(bit)

        pub = open("key.pub", "w")
        pub.write(public_key)

        pri = open("key.pri", "w")
        pri.write(private_key)
        

    def encrypt(self, message, public_key):
        y = Utils.b64_to_dec(public_key['y'])
        g = Utils.b64_to_dec(public_key['g'])
        p = Utils.b64_to_dec(public_key['p'])

        p_key = { 'y': y, 'g': g, 'p': p }

        encrypted = ''
        for m in message:
            k = random.randint(1, p-2)
            res = self.elgamal.encrypt(ord(m), k, p_key)
            a = Utils.dec_to_b64(res['a'])
            b = Utils.dec_to_b64(res['b'])
            encrypted += a + b
        return encrypted

    def decrypt(self, encrypted, private_key):
        x = Utils.b64_to_dec(private_key['x'])
        p = Utils.b64_to_dec(private_key['p'])

        p_key = { 'x': x, 'p': p }
        
        encrypted = encrypted.split('=\n')[:-1]
        encrypted_paired = []
        for i in range(0, len(encrypted), 2):
            a = Utils.b64_to_dec(encrypted[i] + '=\n')
            b = Utils.b64_to_dec(encrypted[i+1] + '=\n')
            encrypted_paired.append({ 'a': a, 'b': b })

        message = ''
        for m in encrypted_paired:
            message += chr(self.elgamal.decrypt(m, p_key))
        return message

In [222]:
machine = ElGamalMachine()

public_key, private_key = machine.create_key(256)

print(public_key)
print(private_key)

# machine.create_key_file(256)

b+YaMc8gLQceKDLRUMSJfFezZZTaT+mOY/fGOAcB+gE=
Vb+7PX1cGT35RpxvHo9G2KZjWpv6lmDoLOywdv0b5iI=
wWMuo4Cyz8oZP+DUg+9gtIsFmtSbukC6Q6UyGoaw7Gc=

ASuQb7kGjl0GWY7+clb+wG0h90GeDEyK3fhaIUkckig=
wWMuo4Cyz8oZP+DUg+9gtIsFmtSbukC6Q6UyGoaw7Gc=



In [126]:
class EllipticPoint:
    def __init__(self, x=0, y=0, inf=False):
        self.x = x if not inf else math.inf
        self.y = y if not inf else math.inf
        self.inf = inf

    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def is_infinity(self):
        return self.inf
    
    def mirror(self):
        return EllipticPoint(self.x, -self.y, self.inf)

    def __eq__(self, point):
        if(isinstance(point, EllipticPoint)):
            if(self.inf):
                return self.inf == point.inf
            else:
                return self.x == point.x and self.y == point.y
            
        return False

    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)

In [187]:
class EllipticCurve:
    def __init__(self, a, b, p):
        ''' y^2 = x^3 + ax + b '''
        self.a = a
        self.b = b
        self.p = p
        self.points = []
        if(self.is_singular()):
            print("Error: Curve is Singular!.")
    
    def is_singular(self):
        return((((4 * (self.a ** 3)) + (27 * (self.b ** 2))) % self.p) == 0)

    def get_points(self):
        points = [EllipticPoint(inf=True)]
        
        for x in range(self.p):
            y_square = x ** 3 + self.a * x + self.b

            for point in Utils.mod_sqrt(y_square % self.p, self.p):
                points.append(EllipticPoint(x, point))
        self.points = points 
    
    def get_y_square(self, x):
        y_square = x ** 3 + self.a * x + self.b

        for point in Utils.mod_sqrt(y_square % self.p, self.p):
            return(EllipticPoint(x, point))

    def point_addition(self, P, Q):
        if(P == Q):
            return self.point_multiplication(P)
        else:
            xp = P.get_x(); yp = P.get_y();
            xq = Q.get_x(); yq = Q.get_y();

            if(xp == xq):
                return EllipticPoint(inf=True)
            else:
                m = ((yp-yq) * pow((xp-xq), -1, self.p)) % self.p

                xr = (m ** 2 - xp - xq) % self.p
                yr = (m * (xp - xr) - yp) % self.p
                return EllipticPoint(xr, yr)

    def point_substraction(self, P, Q):
        return self.point_addition(P, EllipticPoint(Q.get_x(), -Q.get_y() % self.p))

    def point_multiplication(self, P):
        
        xp = P.get_x(); yp = P.get_y();

        if(yp == 0):
            return EllipticPoint(inf=True)
        else:
            m = ((3 * (xp ** 2) + self.a) * pow((2 * yp), -1, self.p)) % self.p
            xr = (m ** 2 - 2 * xp) % self.p
            yr = (m * (xp - xr) - yp) % self.p
            return EllipticPoint(xr, yr)

    def point_scalar_multiplication(self, P, k):
        if(k == 1):
            return P
        elif(k == 2):
            return self.point_multiplication(P)
        else:
            prev = P
            res = self.point_multiplication(P)
            for i in range(k-2):
                if(res.is_infinity()):
                    res = self.point_addition(prev, self.point_multiplication(P))
                else:
                    prev = res
                    res = self.point_addition(res, P)
    
            return res
    

In [201]:
curve = EllipticCurve(1, 6, 11)
curve.get_points()


P = EllipticPoint(8, 8)

for i in range(1, len(curve.points)):
    print(curve.point_scalar_multiplication(P, i))



P = EllipticPoint(3, 5)
Q = EllipticPoint(5, 9)
print()
print(curve.point_addition(P, Q))


(8, 8)
(7, 2)
(10, 2)
(2, 7)
(5, 9)
(3, 5)
(3, 6)
(5, 2)
(2, 4)
(10, 9)
(7, 9)
(8, 3)

(7, 9)


In [190]:
class ECCElGamalMachine:
    def __init__(self):
        pass

    def is_prime(self, num, test_count=1000):
        if(num == 1):
            return False
        if(test_count >= num):
            test_count = num - 1
        for _ in range(test_count):
            val = random.randint(1, num - 1)
            if(pow(val, num-1, num) != 1):
                return False
        return True
    def create_random_prime(self, bit):
        found = False
        while not found:
            p = random.randint(2**(bit-1)+1, 2**bit-1)
            if self.is_prime(p):
                return p
    
    def create_agreement(self, bit):
        p = self.create_random_prime(bit)

        curve_a = random.randint(-10, 10)
        curve_b = random.randint(-200, 200)

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        found = False

        B = None

        while(not found):
            B = curve.points[random.randint(1, len(curve.points) - 2)]
            if(not B.is_infinity()):
                found = True
        k = 2

        return p, curve_a, curve_b, B, k

    def create_key(self, p, curve_a, curve_b, B):

        pri = random.randint(1, p-2)

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()
       
        public_key = curve.point_scalar_multiplication(B, pri)
        private_key = pri

        return public_key, private_key

    def point_available(self, points, x):
        res = None
        for point in points:
            if(point.get_x() == x):
                res = point
                break
        return res

    def encode(self, message, p, curve_a, curve_b, k):

        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        encoded = []

        for m in message:
            m = ord(m)
            
            point = curve.points[k*m + 1]

            encoded.append(point)  
        return encoded

    def decode(self, encrypted, p, curve_a, curve_b, k):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()
        points = curve.points
        decoded = ""

        for e in encrypted:
            for i in range(len(points)):
                if(points[i].get_x() == e.get_x()):
                    break
            decoded += chr(math.floor((i - 1) / k))
        return decoded

    def encrypt(self, message, public_key, p, curve_a, curve_b, B, k):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        encoded = self.encode(message, p, curve_a, curve_b, k)

        print("ENCODED")
        for e in encoded:
            print(e)

        encrypted = []

        for encode in encoded:
            found = False
            while(not found):
                random_k = random.randint(1, p-1)
                kB = curve.point_scalar_multiplication(B, random_k)
                PmkPb = curve.point_addition(encode, curve.point_scalar_multiplication(public_key, random_k))
                if(not kB.is_infinity() and not PmkPb.is_infinity()):
                    found = True
            
            encrypted.append({ 'kB': kB, 'PmkPb': PmkPb })
            print("kB\t:" + str(kB))
            print("PmkPb\t:" + str(PmkPb))

        return encrypted

    def decrypt(self, encrypted, private_key, p, curve_a, curve_b):
        curve = EllipticCurve(curve_a, curve_b, p)
        curve.get_points()

        decrypted = []

        for e in encrypted:
            print("PmkPb\t: " + str(e['PmkPb']))
            print("Mul\t: " + str(curve.point_scalar_multiplication(e['kB'], private_key)))
            decrypted.append(curve.point_substraction(e['PmkPb'], curve.point_scalar_multiplication(e['kB'], private_key)))

        print("DECRYPTED")
        for e in decrypted:
            print(e)
        decoded = self.decode(decrypted, p, curve_a, curve_b, k)

        return decoded
            


In [203]:
machine = ECCElGamalMachine()

p, curve_a, curve_b, B, k = machine.create_agreement(12)

print()

print("GENERATE RANDOM AGREEMENT")
print("p\t: " + str(p))
print("crv_a\t: " + str(curve_a))
print("crv_b\t: " + str(curve_b))
print("B\t: " + str(B))
print("k\t: " + str(k))

print()

alice_public_key, alice_private_key = machine.create_key(p, curve_a, curve_b, B)
print("GENERATE ALICE KEYS")
print("public_key\t: " + str(alice_public_key))
print("private_key\t: " + str(alice_private_key))

print()

bob_public_key, bob_private_key = machine.create_key(p, curve_a, curve_b, B)
print("GENERATE BOB KEYS")
print("public_key\t: " + str(bob_public_key))
print("private_key\t: " + str(bob_private_key))

print()

print("ENCRYPTED")
encrypted = machine.encrypt("AUFA Fadhlurohman", bob_public_key, p, curve_a, curve_b, B, k)
print(encrypted)

print()

print("DECRYPTED")
decrypted = machine.decrypt(encrypted, bob_private_key, p, curve_a, curve_b)
print(decrypted)


N: 2297

GENERATE RANDOM AGREEMENT
p	: 2351
crv_a	: -1
crv_b	: 177
B	: (1160, 17)
k	: 2

GENERATE ALICE KEYS
public_key	: (872, 236)
private_key	: 1366

GENERATE BOB KEYS
public_key	: (875, 1666)
private_key	: 588

ENCRYPTED
ENCODED
(130, 293)
(170, 135)
(137, 381)
(130, 293)
(55, 359)
(137, 381)
(190, 775)
(194, 496)
(200, 251)
(208, 680)
(231, 555)
(219, 805)
(212, 912)
(200, 251)
(210, 248)
(190, 775)
(211, 352)
kB	:(1451, 883)
PmkPb	:(23, 111)
kB	:(474, 23)
PmkPb	:(2162, 1399)
kB	:(2259, 886)
PmkPb	:(1068, 56)
kB	:(920, 2211)
PmkPb	:(1923, 725)
kB	:(1198, 2064)
PmkPb	:(710, 1552)
kB	:(354, 2032)
PmkPb	:(1262, 2275)
kB	:(1710, 730)
PmkPb	:(890, 801)
kB	:(2157, 2154)
PmkPb	:(1012, 2258)
kB	:(2269, 685)
PmkPb	:(2206, 276)
kB	:(725, 390)
PmkPb	:(174, 1011)
kB	:(236, 1473)
PmkPb	:(978, 1045)
kB	:(980, 1618)
PmkPb	:(2124, 1236)
kB	:(1876, 749)
PmkPb	:(1923, 725)
kB	:(253, 139)
PmkPb	:(2222, 1963)
kB	:(2288, 1970)
PmkPb	:(1066, 1562)
kB	:(765, 1320)
PmkPb	:(1537, 1767)
kB	:(1615, 1912)
Pm

In [ ]:
# ALICE
# Kunci Privat = a
# Kunci Publik = PA = a . B

# BOB
# Kunci Privat = b
# Kunci Publik = PB = b . B

# Bilangan Acak k dalam interval [1, p-1]

# Ciphertext(PM) = PC = [(kB), (PM + kPB)]
# b . (kB) 
